<a href="https://colab.research.google.com/github/thomaskarsten90/praticas_python/blob/main/ETL_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import unicodedata
import re

# ====================================
# 1. Criar um CSV de exemplo (dados sujos)
# ====================================
dados = {
    "id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "nacionalidade": [
        "BrasileiraRa",
        "Brasileiroh",
        "Brasileiro⁸",
        "BRSILEIRO",
        "Brasileiro2",
        "BRASILEIRO ",
        "brasileiro",
        "COLOMBIANO",
        "colombia",
        "Venezuelano"
    ]
}

df_raw = pd.DataFrame(dados)
df_raw.to_csv("nacionalidade_raw.csv", index=False, encoding="utf-8")
print(" CSV sujo criado: nacionalidade_raw.csv")
print(df_raw.head())


 CSV sujo criado: nacionalidade_raw.csv
   id nacionalidade
0   1  BrasileiraRa
1   2   Brasileiroh
2   3   Brasileiro⁸
3   4     BRSILEIRO
4   5   Brasileiro2


In [2]:
# ====================================
# 2. Funções de limpeza
# ====================================
def remover_acentos(texto: str) -> str:
    """Remove acentos e normaliza unicode."""
    if pd.isna(texto):
        return ""
    nfkd = unicodedata.normalize("NFKD", texto)
    return "".join([c for c in nfkd if not unicodedata.combining(c)])

def limpar_string(texto: str) -> str:
    """Remove números, caracteres estranhos e espaços extras."""
    if pd.isna(texto):
        return ""
    texto = texto.upper().strip()  # maiúsculas + tira espaços
    texto = remover_acentos(texto) # tira acentos
    texto = re.sub(r"[^A-Z ]", "", texto)  # mantém só letras e espaço
    return texto

def padronizar_nacionalidade(texto: str) -> str:
    """Padroniza nacionalidades conhecidas."""
    texto = limpar_string(texto)
    if "BRASIL" in texto or "BRASILEIR" in texto:
        return "BRASILEIRO"
    elif "COLOMB" in texto:
        return "COLOMBIANO"
    elif "VENEZ" in texto:
        return "VENEZUELANO"
    else:
        return "OUTROS"


In [3]:
# ====================================
# 3. Mini ETL (Extract, Transform, Load)
# ====================================

# Extract
df = pd.read_csv("nacionalidade_raw.csv")

# Transform
df["nacionalidade_limpa"] = df["nacionalidade"].apply(padronizar_nacionalidade)

# Load
df.to_csv("nacionalidade_clean.csv", index=False, encoding="utf-8")

print(" ETL finalizado! Arquivo salvo: nacionalidade_clean.csv")
print(df.head())

 ETL finalizado! Arquivo salvo: nacionalidade_clean.csv
   id nacionalidade nacionalidade_limpa
0   1  BrasileiraRa          BRASILEIRO
1   2   Brasileiroh          BRASILEIRO
2   3   Brasileiro⁸          BRASILEIRO
3   4     BRSILEIRO              OUTROS
4   5   Brasileiro2          BRASILEIRO
